# LSTMs for Human Activity Recognition

Human activity recognition using smartphones dataset and an LSTM RNN. Classifying the type of movement amongst six categories:
- WALKING,
- WALKING_UPSTAIRS,
- WALKING_DOWNSTAIRS,
- SITTING,
- STANDING,
- LAYING.

Compared to a classical approach, using a Recurrent Neural Networks (RNN) with Long Short-Term Memory cells (LSTMs) require no or almost no feature engineering. Data can be fed directly into the neural network who acts like a black box, modeling the problem correctly. Other research on the activity recognition dataset used mostly use a big amount of feature engineering, which is rather a signal processing approach combined with classical data science techniques. The approach here is rather very simple in terms of how much did the data was preprocessed. 

## Video dataset overview

Follow this link to see a video of the 6 activities recorded in the experiment with one of the participants:

<p align="center">
  <a href="http://www.youtube.com/watch?feature=player_embedded&v=XOEN9W05_4A
" target="_blank"><img src="http://img.youtube.com/vi/XOEN9W05_4A/0.jpg" 
alt="Video of the experiment" width="400" height="300" border="10" /></a>
  <a href="https://youtu.be/XOEN9W05_4A"><center>[Watch video]</center></a>
</p>

## Details about input data

I will be using an LSTM on the data to learn (as a cellphone attached on the waist) to recognise the type of activity that the user is doing. The dataset's description goes like this:

> The sensor signals (accelerometer and gyroscope) were pre-processed by applying noise filters and then sampled in fixed-width sliding windows of 2.56 sec and 50% overlap (128 readings/window). The sensor acceleration signal, which has gravitational and body motion components, was separated using a Butterworth low-pass filter into body acceleration and gravity. The gravitational force is assumed to have only low frequency components, therefore a filter with 0.3 Hz cutoff frequency was used. 

That said, I will use the almost raw data: only the gravity effect has been filtered out of the accelerometer  as a preprocessing step for another 3D feature as an input to help learning. 

## What is an RNN?

As explained in [this article](http://karpathy.github.io/2015/05/21/rnn-effectiveness/), an RNN takes many input vectors to process them and output other vectors. It can be roughly pictured like in the image below, imagining each rectangle has a vectorial depth and other special hidden quirks in the image below. **In our case, the "many to one" architecture is used**: we accept time series of feature vectors (one vector per time step) to convert them to a probability vector at the output for classification. Note that a "one to one" architecture would be a standard feedforward neural network. 

<img src="http://karpathy.github.io/assets/rnn/diags.jpeg" />

An LSTM is an improved RNN. It is more complex, but easier to train, avoiding what is called the vanishing gradient problem. 


## Results 

Scroll on! Nice visuals awaits. 

In [1]:
#loader class for loading and preprocessing the data sets
from loader import Loader
import time

# All Includes


import matplotlib
import matplotlib.pyplot as plt
from sklearn import metrics

import numpy as np
import tensorflow as tf
import keras
import os
import pickle
import pandas as pd

from keras.models import Sequential, Input, Model
from keras.layers.recurrent import LSTM
from keras import layers
from keras.layers import Activation, Dense, Flatten, BatchNormalization, Reshape, Dropout, Conv1D, MaxPool1D
from keras.optimizers import Adam, SGD
from keras.utils import np_utils
from keras.layers.wrappers import TimeDistributed, Bidirectional
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import normalize
from keras import backend as K
from keras.regularizers import l2

np.random.seed(1337)
tf.set_random_seed(1339)

session_conf = tf.ConfigProto(log_device_placement=True)
# # Avoid full memory allocation on GPU
session_conf.gpu_options.allow_growth=True

sess = tf.InteractiveSession(config=session_conf)
K.set_session(sess)

Using TensorFlow backend.


## Selecting a dataset and preparing it:

1) Run the appropriate loader from Loader  class <br>
2) Run the appropriate preprocessor (sliding window size , overlay) <br>
3) Load the pickled output file of the loader <br>

Could potentially be used in a loop for automated testing

In [9]:
load = Loader()

sliding_window = 5
overlay = 1


#load.ward1_load()
#load.ward1_preprocess(sliding_window, overlay) # 40,20
#path = "WARD1"
#load.opportunity_load()
#load.opportunity_preprocess(sliding_window, overlay) # 300, 80
#path = "OpportunityUCIDataset"
#load.daily_sports_load()
#load.daily_sports_preprocess(sliding_window, overlay)  # 25, 15
#path = "Daily and sports activities"
#load.pamap2_load()
#load.pamap2_preprocess(sliding_window,  overlay)  # 270, 30
#path = "PAMAP2"

#load.chiron_first_load()
#frame = load.chiron_first_preprocess(sliding_window, overlay)  # 10,5
#path = "Chiron first"
#load.chiron_second_load()
#frame = load.chiron_second_preprocess(sliding_window, overlay)  # 1,0
path = "Chiron second"


#df = pickle.load(open('WARD1/processed_data.txt','rb')) # -> works
#df = pickle.load(open('OpportunityUCIDataset/full_preprocessed_dataset.txt','rb'))
#df = pickle.load(open('Daily and sports activities/full_preprocessed.txt','rb'))
#df = pickle.load(open('PAMAP2/full_preprocessed.txt','rb'))

#df = pickle.load(open('Chiron first/Chiron_full_preprocessed.txt' ,'rb'))
df = pickle.load(open('Chiron second/Chiron_full_preprocessed.txt' ,'rb'))



In [16]:
#print(df)
print("------------------------------------------------------")
print(df.Activity.value_counts())


------------------------------------------------------
7.0     7806
9.0     4612
1.0     4464
32.0    3637
11.0    3152
12.0    3140
16.0    3116
17.0    2637
3.0     2515
26.0    2233
13.0    2180
8.0     2091
31.0    2047
25.0    2030
5.0     1818
4.0     1723
18.0    1679
14.0    1597
27.0    1457
2.0     1310
10.0    1276
28.0    1264
30.0    1167
15.0    1072
29.0     974
23.0     876
6.0      850
24.0     740
20.0     431
22.0     399
21.0     283
19.0     186
Name: Activity, dtype: int64


## Preparing the leave one out method:

Using methods that prepare the next leave one out division <br>

In [3]:

encoder = LabelEncoder()
encoder.fit(df.values[:,0])

classes = len(encoder.classes_)
class_array = (np.array(df.Activity.unique()).astype(int) - 1)
class_array = [i in range(0,class_array.shape[1])]

def train_test(data, index):
    #training and test sets -> leave one out
    leave_out = df.index.isin([index])

    X_train = df[~leave_out].values[:,2:]
    y_train = df[~leave_out].values[:,0]

    X_test = df[leave_out].values[:,2:]
    y_test = df[leave_out].values[:,0]

    #--------------------------------------------------------------

    #normalization
    X_test = normalize(X_test)
    X_train = normalize(X_train)
    
    #--------------------------------------------------------------
    
    #reshaping to fit keras
    X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
    X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
    
    return X_train, y_train, X_test, y_test

In [4]:
print(class_array)

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25
 26 27 28 29 30 31 32]


## Additionnal Parameters:

Here are some core parameter definitions for the training. 

The whole neural network's structure could be summarised by enumerating those parameters and the fact an LSTM is used. 

In [5]:
def additional_parameters(X_train, X_test):

    training_data_count = X_train.shape[0]  # 21522 training series (with 50% overlap between each serie)
    test_data_count = X_test.shape[0]  # 7794 testing series

    n_steps = X_train.shape[1]  # 128 timesteps per series
    n_input = len(X_train[0][0])  # 7 input parameters per timestep

    print("Steps: ",n_steps, "N imput",n_input)
    
    return training_data_count, test_data_count, n_steps, n_input

In [6]:
# Input Data 

# LSTM Neural Network's internal structure

n_hidden = 32 # Hidden layer num of features
n_classes = classes # Total classes (should go up, or should go down)

# Training 

learning_rate = 0.0025
lambda_loss_amount = 0.0015
training_iters = 300  # Loop 300 times on the dataset
batch_size = 1500
display_iter = 30000  # To show test set accuracy during training


# Some debugging info

print("Some useful info to get an insight on dataset's shape and normalisation:")
print("(X shape, y shape, every X's mean, every X's standard deviation)")
#print(X_test.shape, y_test.shape, np.mean(X_test), np.std(X_test)) # -> takes too long
print("The dataset is therefore properly normalised, as expected, but not yet one-hot encoded.")


Some useful info to get an insight on dataset's shape and normalisation:
(X shape, y shape, every X's mean, every X's standard deviation)
The dataset is therefore properly normalised, as expected, but not yet one-hot encoded.


In [7]:
def one_hot(y_):
    #encoder = LabelEncoder()
    y_encoded = encoder.transform(y_.ravel())#fit_transform(y_.ravel())
    return np_utils.to_categorical(y_encoded, classes)

## Basic regression network

1) Get training and test data sets and calculate some additional parameters <br>
2) Use the additional parameters to build a model <br>
3) Train, test and evaluate the model <br>

In [ ]:
#TODO -> add method calls and a for loop for leave one out

temp = 0 # for printing the current subject
num_epochs = 75 # for convenience
results_acc = np.array([[]]).reshape(num_epochs,0)
results_loss = np.array([[]]).reshape(num_epochs,0)
results_f1 = []


for index in df.index.unique():

    #exception - subject does not contain all possible classes
    
    
    #-------------------------------------------------------------
    #get training and test sets

    X_train, y_train, X_test, y_test = train_test(df, index)

    training_data_count, test_data_count, n_steps, n_input = additional_parameters(X_train, X_test)
    training_iters = training_data_count * training_iters

    y_train = one_hot(y_train)
    y_test = one_hot(y_test)
    
    print(y_train.shape)
    print(y_test.shape)


    #-------------------------------------------------------------
    #make a model

    input = Input(shape=(n_steps, n_input), name="input")
    x = Flatten()(input)
    x = Dense(n_hidden, activation="relu", name="fc1")(x)
    x = Dense(n_hidden, activation="relu", name="fc2")(x)
    x = Dense(n_classes, activation="softmax", name="prediction")(x)
    model = Model(input,x)
    #model.summary()

    #------------------------------------------------------------
    #choose an activation function

    adam = Adam(lr=learning_rate)
    model.compile(optimizer=adam,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    #-------------------------------------------------------------
    #fit a model

    temp += 1
    print('Started fitting model (leave out subject: ' + str(int(index)) + ') ....... ' + str(temp))
    
    t0 = time.time()
    
    hist = model.fit(X_train, y_train, batch_size=batch_size, epochs = num_epochs, validation_data=(X_test, y_test), verbose=0)
    
    
    #-------------------------------------------------------------
    #gather results
    
    #Accuracy and loss
    acc = np.array([hist.history['val_acc']]).reshape(len(hist.history['val_acc']),1)
    loss = np.array([hist.history['val_loss']]).reshape(len(hist.history['val_loss']),1)
    
    results_acc = np.concatenate([results_acc, acc], axis = 1)
    results_loss = np.concatenate([results_loss, loss], axis = 1)
    
    #Confusion matrix
    y_true = np.argmax(y_test, axis=1)
    y_pred = model.predict(X_test)
    y_pred = np.argmax(y_pred, axis=1)

    conf = pd.DataFrame(metrics.confusion_matrix(y_true, y_pred, labels = class_array))
    conf.to_csv(path + "/Results/Basic regression network/Confidence_matrix_S" + str(int(index)) + '_(' + str(sliding_window) + ',' + str(overlay) + ").csv", index=False)
    
    #F1 score
    results_f1.append(metrics.f1_score(y_true, y_pred, average='micro'))
    
    print('')
    print('Confusion matrix:')
    print(conf)
    print('F1 score ....... ' + str(results_f1[temp-1]))
    print('')
    
    t1 = time.time()

    print('Finished fitting model (leave out subject: ' + str(int(index)) + ') ....... ' + str(temp))
    print("Total time: " + str(t1-t0))
    print('------------------------------------------------------------')


Steps:  181 N imput 1
(58858, 32)
(5904, 32)
Started fitting model (leave out subject: 1) ....... 1

Confusion matrix:
     0    1   2    3   4    5    6   7    8    9  ...  22  23  24  25  26  27  \
0   155    0   0    0   0    3    0   0    0    0 ...   0   0   0   0   0   0   
1     0  282   0    0   0    7    0   0    0   13 ...   0   0   0   0   0   0   
2     0    0  53    0   0  210    0   0    0    0 ...   0   0   0   0   0   0   
3     0    0   0  194   0    0    0   0    0    0 ...   0   0   0   0   0   0   
4     0    0   0    0  18  126    0   0    0    0 ...   0   0   0   0   0   0   
5     0    0   0    0   1  828    0   0    0    0 ...   0   0   0   0   0   3   
6    36    1   0    0   0    0  222   0    0    0 ...   0   0   0   0   0   0   
7     0    0   0    0   0    0    0  85   29    0 ...   0   0   0   0   0   0   
8     0    0   0    0   8    5    0   0  131    0 ...   0   0   0   0   0   0   
9     0    4   1    3   0   23   24   0    0  330 ...   0   0   0   0  

In [10]:
#print some results
print('Average accuracy ...... ' + str( np.sum(np.sum(results_acc, axis=0)/num_epochs)/temp) )
print('Average loss .......... ' + str( np.sum(np.sum(results_loss, axis=0)/num_epochs)/temp) )
print('Average F1 score ...... ' + str( sum(results_f1)/temp) )


Average accuracy ...... 0.595815266404
Average loss .......... 1.37953065779
Average F1 score ...... 0.0
     0    1    2    3   4    5   6   7    8    9  ...  22  23  24  25  26  27  \
0   155    0    0    0   0    3   0   0    0    0 ...   0   0   0   0   0   0   
1     0  312    0    0   0    0   0   0    0    0 ...   0   0   0   0   0   0   
2     0    0  164    0   0  109   0   0    0    0 ...   0   0   0   0   0   0   
3     0    0    0  191   0    0   0   0    0    0 ...   0   0   0   0   0   0   
4     0    0    0    0  43   68   0   0   14    0 ...   0   0   0   0   0   0   
5     0    0    5    0  23  759   0   0    0    0 ...   0   0   0   0   0   4   
6    39    0    0    0   0    0  86   0    0    0 ...   0   0   0   0   0   0   
7     0    0    0    0   0   24   0  42   59    0 ...   0   0   0   0   0   0   
8     0    0    0    0   2   13   0   0  131    0 ...   0   0   0   0   0   0   
9     0    0    0    3   2   25  17   0    0  268 ...   0   0   0   0   0   0   
10  

In [23]:
#add some labels and save the results to csv

#Accuracy and loss:
result_acc = pd.DataFrame(results_acc, columns = df.index.unique())
result_loss = pd.DataFrame(results_loss, columns = df.index.unique())

result_acc.to_csv(path + "/Results/Basic regression network/Accuracy" + "_(" + str(sliding_window) + "," + str(overlay) + ").csv", index=False)
result_loss.to_csv(path + "/Results/Basic regression network/Loss" + "_(" + str(sliding_window) + "," + str(overlay) + ").csv", index=False)

#F1 score:
result_f1 = pd.DataFrame(np.array([results_f1]), columns = df.index.unique().values.astype(int))
result_f1.to_csv(path + "/Results/Basic regression network/F1" + "_(" + str(sliding_window) + "," + str(overlay) + ").csv", index=False)

## Basic LSTM network

In [ ]:
temp = 0 # for printing the current subject
num_epochs = 50 # for convenience
results_acc = np.array([[]]).reshape(num_epochs,0)
results_loss = np.array([[]]).reshape(num_epochs,0)
results_f1 = []

for index in df.index.unique():

    #exception - subject does not contain all possible classes
    
    
    #-------------------------------------------------------------
    #get training and test sets

    X_train, y_train, X_test, y_test = train_test(df, index)

    training_data_count, test_data_count, n_steps, n_input = additional_parameters(X_train, X_test)
    training_iters = training_data_count * training_iters

    y_train = one_hot(y_train)
    y_test = one_hot(y_test)
    
    print(y_train.shape)
    print(y_test.shape)


    #-------------------------------------------------------------
    #make a model
    
    input = Input(shape=(n_steps, n_input), name="input")
    x = Dense(n_hidden, activation="relu", name="fc1")(input)
    x = LSTM(n_hidden, return_sequences=True, name='lstm_1')(x)
    x = LSTM(n_hidden, name='lstm_2')(x)
    x = Dense(n_classes, activation="softmax", activity_regularizer=l2(lambda_loss_amount), name="prediction")(x)
    model = Model(input,x)
    model.summary()
    
    #------------------------------------------------------------
    #choose an activation function

    adam = Adam(lr=learning_rate)
    model.compile(optimizer=adam,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    
    #-------------------------------------------------------------
    #train a model

    temp += 1
    print('Started fitting model (leave out subject: ' + str(index) + ') ....... ' + str(temp))
    
    t0 = time.time()
    
    hist = model.fit(X_train, y_train, batch_size=batch_size, epochs = num_epochs, validation_data=(X_test, y_test), verbose=0) 

    
    #-------------------------------------------------------------
    #gather results
    
    #Accuracy and loss
    acc = np.array([hist.history['val_acc']]).reshape(len(hist.history['val_acc']),1)
    loss = np.array([hist.history['val_loss']]).reshape(len(hist.history['val_loss']),1)
    
    results_acc = np.concatenate([results_acc, acc], axis = 1)
    results_loss = np.concatenate([results_loss, loss], axis = 1)
    
    #Confusion matrix
    y_true = np.argmax(y_test, axis=1)
    y_pred = model.predict(X_test)
    y_pred = np.argmax(y_pred, axis=1)

    conf = pd.DataFrame(metrics.confusion_matrix(y_true, y_pred, labels = class_array))
    conf.to_csv(path + "/Results/Basic LSTM network/Confidence_matrix_S" + str(int(index)) + '_(' + str(sliding_window) + ',' + str(overlay) + ").csv", index=False)
    
    #F1 score
    results_f1.append(metrics.f1_score(y_true, y_pred, average='micro'))
    
    print('')
    print('Confusion matrix:')
    print(conf)
    print('F1 score ....... ' + str(results_f1[temp-1]))
    print('')
    
    t1 = time.time()

    print('Finished fitting model (leave out subject: ' + str(int(index)) + ') ....... ' + str(temp))
    print("Total time: " + str(t1-t0))
    print('------------------------------------------------------------')

##############################################
################# Results ####################

#print some results

print('Average accuracy ...... ' + str( np.sum(np.sum(results_acc, axis=0)/num_epochs)/temp) )
print('Average loss .......... ' + str( np.sum(np.sum(results_loss, axis=0)/num_epochs)/temp) )
print('Average F1 score ...... ' + str( sum(results_f1)/temp) )

#add some labels and save the results to csv

#Accuracy and loss:
result_acc = pd.DataFrame(results_acc, columns = df.index.unique())
result_loss = pd.DataFrame(results_loss, columns = df.index.unique())

result_acc.to_csv(path + "/Results/Basic LSTM network/Accuracy" + "_(" + str(sliding_window) + "," + str(overlay) + ").csv", index=False)
result_loss.to_csv(path + "/Results/Basic LSTM network/Loss" + "_(" + str(sliding_window) + "," + str(overlay) + ").csv", index=False)

#F1 score:
result_f1 = pd.DataFrame(np.array([results_f1]), columns = df.index.unique().values.astype(int))
result_f1.to_csv(path + "/Results/Basic LSTM network/F1" + "_(" + str(sliding_window) + "," + str(overlay) + ").csv", index=False) 

    

Steps:  1000 N imput 1
(25277, 13)
(1456, 13)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 1000, 1)           0         
_________________________________________________________________
fc1 (Dense)                  (None, 1000, 32)          64        
_________________________________________________________________
lstm_1 (LSTM)                (None, 1000, 32)          8320      
_________________________________________________________________
lstm_2 (LSTM)                (None, 32)                8320      
_________________________________________________________________
prediction (Dense)           (None, 13)                429       
Total params: 17,133
Trainable params: 17,133
Non-trainable params: 0
_________________________________________________________________
Started fitting model (leave out subject: 1.0) ....... 1
Train on 25277 samples, validate on

## Basic bidirectional stacked LSTMs

In [53]:
temp = 0 # for printing the current subject
num_epochs = 50 # for convenience
results_acc = np.array([[]]).reshape(num_epochs,0)
results_loss = np.array([[]]).reshape(num_epochs,0)
results_f1 = []


for index in df.index.unique():

    #exception - subject does not contain all possible classes
    
    
    #-------------------------------------------------------------
    #get training and test sets

    X_train, y_train, X_test, y_test = train_test(df, index)

    training_data_count, test_data_count, n_steps, n_input = additional_parameters(X_train, X_test)
    training_iters = training_data_count * training_iters

    y_train = one_hot(y_train)
    y_test = one_hot(y_test)
    
    print(y_train.shape)
    print(y_test.shape)


    #-------------------------------------------------------------
    #make a model
    
    input = Input(shape=(n_steps, n_input), name="input")
    x = Dense(n_hidden, activation="relu", name="fc1")(input)
    x = Bidirectional(LSTM(n_hidden, return_sequences=True, name='lstm_1'))(x)
    x = Bidirectional(LSTM(n_hidden, name='lstm_2'))(x)
    x = Dense(n_classes, activation="softmax", activity_regularizer=l2(lambda_loss_amount), name="prediction")(x)
    model = Model(input,x)
    model.summary()
    
       
    #------------------------------------------------------------
    #choose an activation function

    adam = Adam(lr=learning_rate)
    model.compile(optimizer=adam,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    
    #-------------------------------------------------------------
    #train a model
    
    temp += 1
    print('Started fitting model (leave out subject: ' + str(index) + ') ....... ' + str(temp))
    
    t0 = time.time()
    
    hist = model.fit(X_train, y_train, batch_size=batch_size, epochs = num_epochs, validation_data=(X_test, y_test), verbose=0)

    
    #-------------------------------------------------------------
    #gather results
    
    #Accuracy and loss
    acc = np.array([hist.history['val_acc']]).reshape(len(hist.history['val_acc']),1)
    loss = np.array([hist.history['val_loss']]).reshape(len(hist.history['val_loss']),1)
    
    results_acc = np.concatenate([results_acc, acc], axis = 1)
    results_loss = np.concatenate([results_loss, loss], axis = 1)
    
    #Confusion matrix
    y_true = np.argmax(y_test, axis=1)
    y_pred = model.predict(X_test)
    y_pred = np.argmax(y_pred, axis=1)

    conf = pd.DataFrame(metrics.confusion_matrix(y_true, y_pred, labels = class_array))
    conf.to_csv(path + "/Results/Basic bidirectional stacked LSTMs/Confidence_matrix_S" + str(int(index)) + '_(' + str(sliding_window) + ',' + str(overlay) + ").csv", index=False)
    
    #F1 score
    results_f1.append(metrics.f1_score(y_true, y_pred, average='micro'))
    
    print('')
    print('Confusion matrix:')
    print(conf)
    print('F1 score ....... ' + str(results_f1[temp-1]))
    print('')
    
    t1 = time.time()

    print('Finished fitting model (leave out subject: ' + str(int(index)) + ') ....... ' + str(temp))
    print("Total time: " + str(t1-t0))
    print('------------------------------------------------------------')
    
##############################################
################# Results ####################

#print some results

print('Average accuracy ...... ' + str( np.sum(np.sum(results_acc, axis=0)/num_epochs)/temp) )
print('Average loss .......... ' + str( np.sum(np.sum(results_loss, axis=0)/num_epochs)/temp) )
print('Average F1 score ...... ' + str( sum(results_f1)/temp) )

#add some labels and save the results to csv

#Accuracy and loss:
result_acc = pd.DataFrame(results_acc, columns = df.index.unique())
result_loss = pd.DataFrame(results_loss, columns = df.index.unique())

result_acc.to_csv(path + "/Results/Basic bidirectional stacked LSTMs/Accuracy" + "_(" + str(sliding_window) + "," + str(overlay) + ").csv", index=False)
result_loss.to_csv(path + "/Results/Basic bidirectional stacked LSTMs/Loss" + "_(" + str(sliding_window) + "," + str(overlay) + ").csv", index=False)

#F1 score:
result_f1 = pd.DataFrame(np.array([results_f1]), columns = df.index.unique().values.astype(int))
result_f1.to_csv(path + "/Results/Basic bidirectional stacked LSTMs/F1" + "_(" + str(sliding_window) + "," + str(overlay) + ").csv", index=False) 


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 1000, 1)           0         
_________________________________________________________________
fc1 (Dense)                  (None, 1000, 32)          64        
_________________________________________________________________
bidirectional_1 (Bidirection (None, 1000, 64)          16640     
_________________________________________________________________
bidirectional_2 (Bidirection (None, 64)                24832     
_________________________________________________________________
prediction (Dense)           (None, 2)                 130       
Total params: 41,666
Trainable params: 41,666
Non-trainable params: 0
_________________________________________________________________


## 3x3 HAR stacked residual bidirectional LSTMs

In [56]:
learning_rate = 0.001
lambda_loss_amount = 0.005
clip_gradients = 15.0
n_layers_in_highway = 0
n_stacked_layers = 3
n_hidden = 32

In [57]:
temp = 0 # for printing the current subject
num_epochs = 50 # on last layer
results_acc = np.array([[]]).reshape(num_epochs,0)
results_loss = np.array([[]]).reshape(num_epochs,0)
results_f1 = []

for index in df.index.unique():

    #exception - subject does not contain all possible classes
    
    
    #-------------------------------------------------------------
    #get training and test sets

    X_train, y_train, X_test, y_test = train_test(df, index)

    training_data_count, test_data_count, n_steps, n_input = additional_parameters(X_train, X_test)
    training_iters = training_data_count * training_iters

    y_train = one_hot(y_train)
    y_test = one_hot(y_test)
    
    print(y_train.shape)
    print(y_test.shape)


    ################  FIRST  ##################
    
    #-------------------------------------------------------------
    #make a model
    
    input = Input(shape=(n_steps, n_input), name="input")
    x = Dense(32, activation="relu", name="fc1")(input)
    x = Bidirectional(LSTM(16, return_sequences=True), name='bi_lstm1')(x)
    x = Dense(32, activation="relu", name="fc2")(x)
    shortcut = Bidirectional(LSTM(16, return_sequences=True), name='bi_lstm2')(x)
    shortcut = Dense(32, activation="relu", name="fc3")(shortcut)
    x = layers.add([x, shortcut])
    # x = BatchNormalization(name='bn1')(x)

    x = Bidirectional(LSTM(16, return_sequences=True), name='bi_lstm3')(x)
    x = Dense(32, activation="relu", name="fc4")(x)
    shortcut = Bidirectional(LSTM(16, return_sequences=True), name='bi_lstm4')(x)
    shortcut = Dense(32, activation="relu", name="fc5")(shortcut)
    x = layers.add([x, shortcut])
    # x = BatchNormalization(name='bn2')(x)

    x = Bidirectional(LSTM(16, return_sequences=True), name='bi_lstm5')(x)
    x = Dense(32, activation="relu", name="fc6")(x)
    shortcut = Bidirectional(LSTM(16, return_sequences=True), name='bi_lstm6')(x)
    shortcut = Dense(32, activation="relu", name="fc7")(shortcut)
    x = layers.add([x, shortcut])
    # x = BatchNormalization(name='bn2')(x)

    x = Flatten()(x)
    x = Dense(n_classes, activation="softmax", activity_regularizer=l2(lambda_loss_amount), name="prediction")(x)

    model = Model(input,x)
    model.summary()
    
        
       
    #------------------------------------------------------------
    #choose an activation function

    adam = Adam(lr=learning_rate)
    model.compile(optimizer=adam,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    
    #-------------------------------------------------------------
    #train a model
    
    temp += 1
    print('Started fitting model (leave out subject: ' + str(index) + ') ....... ' + str(temp))
    
    t0 = time.time()
    
    model.fit(X_train, y_train, batch_size=batch_size, epochs = 20, validation_data=(X_test, y_test), verbose=0)
    #results.append(model.evaluate(X_test,y_test))
    print('')
    
    t1 = time.time()

    print('Finished fitting model (leave out subject: ' + str(index) + ') ....... ' + str(temp))
    print("Total time: " + str(t1-t0))
    print('------------------------------------------------------------')
    
    
    
    ############## Second ##########################

    
    
    #-------------------------------------------------------------
    #make a model
    
    input = Input(shape=(n_steps, n_input), name="input")
    x = Dense(32, activation="relu", name="fc1")(input)
    x = Bidirectional(LSTM(32, return_sequences=True), name='bi_lstm1')(x)
    x = Bidirectional(LSTM(32, return_sequences=True), name='bi_lstm2')(x)
    x = Bidirectional(LSTM(32, return_sequences=True), name='bi_lstm3')(x)
    x = Dense(32, activation="relu", name="fc2")(x)
    # x = BatchNormalization(name='bn1')(x)

    x = Flatten()(x)
    x = Dense(n_classes, activation="softmax", activity_regularizer=l2(lambda_loss_amount), name="prediction")(x)

    model = Model(input,x)
    model.summary()
       
    #------------------------------------------------------------
    #choose an activation function

    adam = Adam(lr=learning_rate)
    model.compile(optimizer=adam,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    
    #-------------------------------------------------------------
    #train a model
    
    temp += 1
    print('Started fitting model (leave out subject: ' + str(index) + ') ....... ' + str(temp))
    
    t0 = time.time()
    
    model.fit(X_train, y_train, batch_size=batch_size, epochs = 200, validation_data=(X_test, y_test), verbose=0)
    #results.append(model.evaluate(X_test,y_test))
    print('')
    
    t1 = time.time()

    print('Finished fitting model (leave out subject: ' + str(index) + ') ....... ' + str(temp))
    print("Total time: " + str(t1-t0))
    print('------------------------------------------------------------')
    
    
    ################ Third ############################
    
    #-------------------------------------------------------------
    #make a model
    
    input = Input(shape=(n_steps, n_input), name="input")
    x = Dense(32, activation="relu", name="fc1")(input)
    x = Bidirectional(LSTM(32, return_sequences=True), name='bi_lstm1')(x)
    # x = Bidirectional(LSTM(16, return_sequences=True), name='bi_lstm2')(x)
    x = Dense(32, activation="relu", name="fc4")(x)

    x = Flatten()(x)
    x = Dense(n_classes, activation="softmax", activity_regularizer=l2(lambda_loss_amount), name="prediction")(x)

    model = Model(input,x)
    model.summary()
       
    #------------------------------------------------------------
    #choose an activation function

    adam = Adam(lr=learning_rate)
    model.compile(optimizer=adam,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    
    #-------------------------------------------------------------
    #train a model
    
    temp += 1
    print('Started fitting model (leave out subject: ' + str(index) + ') ....... ' + str(temp))
    
    t0 = time.time()
    
    hist = model.fit(X_train, y_train, batch_size=batch_size, epochs = num_epochs, validation_data=(X_test, y_test), verbose=0)

    
    #-------------------------------------------------------------
    #gather results
    
    #Accuracy and loss
    acc = np.array([hist.history['val_acc']]).reshape(len(hist.history['val_acc']),1)
    loss = np.array([hist.history['val_loss']]).reshape(len(hist.history['val_loss']),1)
    
    results_acc = np.concatenate([results_acc, acc], axis = 1)
    results_loss = np.concatenate([results_loss, loss], axis = 1)
    
    #Confusion matrix
    y_true = np.argmax(y_test, axis=1)
    y_pred = model.predict(X_test)
    y_pred = np.argmax(y_pred, axis=1)

    conf = pd.DataFrame(metrics.confusion_matrix(y_true, y_pred, labels = class_array))
    conf.to_csv(path + "/Results/3x3 HAR stacked residual bidirectional LSTMs/Confidence_matrix_S" + str(int(index)) + '_(' + str(sliding_window) + ',' + str(overlay) + ").csv", index=False)
    
    #F1 score
    results_f1.append(metrics.f1_score(y_true, y_pred, average='micro'))
    
    print('')
    print('Confusion matrix:')
    print(conf)
    print('F1 score ....... ' + str(results_f1[temp-1]))
    print('')
    
    t1 = time.time()

    print('Finished fitting model (leave out subject: ' + str(int(index)) + ') ....... ' + str(temp))
    print("Total time: " + str(t1-t0))
    print('------------------------------------------------------------')

    

##############################################
################# Results ####################

#print some results

print('Average accuracy ...... ' + str( np.sum(np.sum(results_acc, axis=0)/num_epochs)/temp) )
print('Average loss .......... ' + str( np.sum(np.sum(results_loss, axis=0)/num_epochs)/temp) )
print('Average F1 score ...... ' + str( sum(results_f1)/temp) )

#add some labels and save the results to csv

#Accuracy and loss:
result_acc = pd.DataFrame(results_acc, columns = df.index.unique())
result_loss = pd.DataFrame(results_loss, columns = df.index.unique())

result_acc.to_csv(path + "/Results/3x3 HAR stacked residual bidirectional LSTMs/Accuracy" + "_(" + str(sliding_window) + "," + str(overlay) + ").csv", index=False)
result_loss.to_csv(path + "/Results/3x3 HAR stacked residual bidirectional LSTMs/Loss" + "_(" + str(sliding_window) + "," + str(overlay) + ").csv", index=False)

#F1 score:
result_f1 = pd.DataFrame(np.array([results_f1]), columns = df.index.unique().values.astype(int))
result_f1.to_csv(path + "/Results/3x3 HAR stacked residual bidirectional LSTMs/F1" + "_(" + str(sliding_window) + "," + str(overlay) + ").csv", index=False) 

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input (InputLayer)               (None, 1000, 1)       0                                            
____________________________________________________________________________________________________
fc1 (Dense)                      (None, 1000, 32)      64          input[0][0]                      
____________________________________________________________________________________________________
bi_lstm1 (Bidirectional)         (None, 1000, 32)      6272        fc1[0][0]                        
____________________________________________________________________________________________________
fc2 (Dense)                      (None, 1000, 32)      1056        bi_lstm1[0][0]                   
___________________________________________________________________________________________

## DeepConvLSTM
<br>
This Part of the code only does a summary of the model, the rest is missing

In [ ]:

input = Input(shape=(n_steps, n_input), name="input")
x = Dense(128, activation="relu", name="fc1")(input)
x = Bidirectional(LSTM(128, return_sequences=True), name='bi_lstm1')(x)
x = Bidirectional(LSTM(128, return_sequences=True), name='bi_lstm2')(x)
x = Dense(128, activation="relu", name="fc4")(x)

x = Flatten()(x)
x = Dense(n_classes, activation="softmax", activity_regularizer=l2(lambda_loss_amount), name="prediction")(x)

model = Model(input,x)
model.summary()


## Basic ConvLSTM

In [ ]:
temp = 0 # for printing the current subject
num_epochs = 50 # for convenience
results_acc = np.array([[]]).reshape(num_epochs,0)
results_loss = np.array([[]]).reshape(num_epochs,0)
results_f1 = []

for index in df.index.unique():

    #exception - subject does not contain all possible classes
    
    
    #-------------------------------------------------------------
    #get training and test sets

    X_train, y_train, X_test, y_test = train_test(df, index)

    training_data_count, test_data_count, n_steps, n_input = additional_parameters(X_train, X_test)
    training_iters = training_data_count * training_iters

    y_train = one_hot(y_train)
    y_test = one_hot(y_test)
    
    print(y_train.shape)
    print(y_test.shape)

    
    #-------------------------------------------------------------
    #make a model

    input = Input(shape=(n_steps, n_input), name="input")
    x = Conv1D(64, 5, activation='relu', name='conv1')(input)
    x = MaxPool1D()(x)
    x = Conv1D(64, 5, activation='relu', name='conv2')(x)
    x = MaxPool1D()(x)
    x = Flatten()(x)
    x = Dropout(.5)(x)
    x = Dense(n_classes, activation="softmax", activity_regularizer=l2(lambda_loss_amount), name="prediction")(x)

    model = Model(input,x)
    model.summary()
    
       
    #------------------------------------------------------------
    #choose an activation function

    adam = Adam(lr=learning_rate)
    model.compile(optimizer=adam,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    
    #-------------------------------------------------------------
    #train a model
    
    temp += 1
    print('Started fitting model (leave out subject: ' + str(index) + ') ....... ' + str(temp))
    
    t0 = time.time()
    
    hist = model.fit(X_train, y_train, batch_size=batch_size, epochs = num_epochs, validation_data=(X_test, y_test), verbose=0)
    
    
    #-------------------------------------------------------------
    #gather results
    
    #Accuracy and loss
    acc = np.array([hist.history['val_acc']]).reshape(len(hist.history['val_acc']),1)
    loss = np.array([hist.history['val_loss']]).reshape(len(hist.history['val_loss']),1)
    
    results_acc = np.concatenate([results_acc, acc], axis = 1)
    results_loss = np.concatenate([results_loss, loss], axis = 1)
    
    #Confusion matrix
    y_true = np.argmax(y_test, axis=1)
    y_pred = model.predict(X_test)
    y_pred = np.argmax(y_pred, axis=1)

    conf = pd.DataFrame(metrics.confusion_matrix(y_true, y_pred, labels = class_array))
    conf.to_csv(path + "/Results/Basic ConvLSTM/Confidence_matrix_S" + str(int(index)) + '_(' + str(sliding_window) + ',' + str(overlay) + ").csv", index=False)
    
    #F1 score
    results_f1.append(metrics.f1_score(y_true, y_pred, average='micro'))
    
    print('')
    print('Confusion matrix:')
    print(conf)
    print('F1 score ....... ' + str(results_f1[temp-1]))
    print('')
    
    t1 = time.time()

    print('Finished fitting model (leave out subject: ' + str(int(index)) + ') ....... ' + str(temp))
    print("Total time: " + str(t1-t0))
    print('------------------------------------------------------------')

##############################################
################# Results ####################

#print some results

print('Average accuracy ...... ' + str( np.sum(np.sum(results_acc, axis=0)/num_epochs)/temp) )
print('Average loss .......... ' + str( np.sum(np.sum(results_loss, axis=0)/num_epochs)/temp) )
print('Average F1 score ...... ' + str( sum(results_f1)/temp) )

#add some labels and save the results to csv

#Accuracy and loss:
result_acc = pd.DataFrame(results_acc, columns = df.index.unique())
result_loss = pd.DataFrame(results_loss, columns = df.index.unique())

result_acc.to_csv(path + "/Results/Basic ConvLSTM/Accuracy" + "_(" + str(sliding_window) + "," + str(overlay) + ").csv", index=False)
result_loss.to_csv(path + "/Results/Basic ConvLSTM/Loss" + "_(" + str(sliding_window) + "," + str(overlay) + ").csv", index=False)

#F1 score:
result_f1 = pd.DataFrame(np.array([results_f1]), columns = df.index.unique().values.astype(int))
result_f1.to_csv(path + "/Results/Basic ConvLSTM/F1" + "_(" + str(sliding_window) + "," + str(overlay) + ").csv", index=False) 

Steps:  72900 N imput 1
(296, 5)
(195, 5)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 72900, 1)          0         
_________________________________________________________________
conv1 (Conv1D)               (None, 72896, 64)         384       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 36448, 64)         0         
_________________________________________________________________
conv2 (Conv1D)               (None, 36444, 64)         20544     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 18222, 64)         0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 1166208)           0         
_________________________________________________________________
dropout_1 (Dropout)          (None